# Preprocessing

Overview of artifact detection
https://mne.tools/stable/auto_tutorials/preprocessing/10_preprocessing_overview.html

## Import

In [ ]:
import os
import pathlib
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import mne
import mne_bids
from mne.preprocessing import (ICA, create_eog_epochs, 
                               compute_proj_eog, create_ecg_epochs, corrmap) 

matplotlib.use('Agg') #Raises no browsers
#matplotlib.use('Qt5Agg') #Raises browsers
mne.set_log_level('warning') #All mne functions will only spit out output if they have reason to.

## Directory

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
epoch_reports = "02_epochs"
preprocessed_reports = "03_preprocessed"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> preprocessed_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, preprocessed_reports)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, preprocessed_reports))

In [ ]:
epoch_report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+epoch_reports)

In [ ]:
preprocessed_report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+preprocessed_reports)

## ICA Analysis

Repairing artifacts with ICA
https://mne.tools/stable/auto_tutorials/preprocessing/40_artifact_correction_ica.html

In [ ]:
print('######################## Computing ICA analysis. ########################')

### Template

#### Input Template

In [ ]:
template_path = mne_bids.BIDSPath(subject='01',
                              session='WM1',
                              task='isi0',
                              datatype='eeg',
                              root=bids_root)

template_raw = mne_bids.read_raw_bids(template_path) 
template_raw.load_data() 

#### Interpolate Bads

In [ ]:
template_raw.info['bads'] = ['E4', 'E5', 'E11', 'E26', 'E27', 'E29', 'E33', 'E34', 'E35', 'E39', 'E40', 'E41', 'E50', 'E54', 'E61', 'E68', 'E73', 'E91', 'E106', 'E107', 'E110', 'E111', 'E116', 'E118', 'E123']


In [ ]:
template_raw = template_raw.copy().interpolate_bads(reset_bads=True)

#### Reset Channel Types: Template

In [ ]:
template_raw = template_raw.copy().pick_types(meg=False, eeg=True, eog=True)
template_raw.set_channel_types({'E8': 'eog'})
template_raw.set_channel_types({'E25': 'eog'})
len(template_raw.ch_names)
template_raw

In [ ]:
template_raw.set_eeg_reference('average',ch_type='eeg',projection=True)  # needed for inverse modeling

#### Pass Template Filter

In [ ]:
high_pass_template = template_raw.filter(l_freq=1, h_freq=None) 

#### Template ICA Analysis

In [ ]:
template_ica = ICA(n_components=15, max_iter='auto', random_state=97)
template_ica.fit(high_pass_template)
template_ica

### Data 

#### Input Data

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject,
                              session=session,
                              task=task,
                              datatype='eeg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path) 
events, event_id = mne.events_from_annotations(raw) 
raw.load_data() 

#### Interpolate Bads

In [ ]:
raw.info['bads'] = bad_list

In [ ]:
raw = raw.copy().interpolate_bads(reset_bads=True)

#### Reset Channel Types: Input

In [ ]:
raw = raw.copy().pick_types(meg=False, eeg=True, eog=True)
raw.set_channel_types({'E8': 'eog'})
raw.set_channel_types({'E25': 'eog'})
len(raw.ch_names)
raw

In [ ]:
raw.set_eeg_reference('average',ch_type='eeg',projection=True)  # needed for inverse modeling

#### Pass Data Filter

In [ ]:
high_pass_raw = raw.filter(l_freq=1, h_freq=None) 

#### Data ICA Analysis

In [ ]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
ica.fit(high_pass_raw)
ica

In [ ]:
ica.plot_components()

## Eye Blinks Rejection

### Reject Eye Blinks Component

In [ ]:
icas = list()
icas.append(template_ica)
icas.append(ica)
icas

In [ ]:
print('Initializing Corrmap; 60% template match.')

In [ ]:
threshold = .60

In [ ]:
corrmap(icas=icas, template=(0, 5), threshold = threshold, label = 'eyeblink', plot = True)

In [ ]:
def run_corrmap():
    global threshold
    threshold += .01
    thresh_name = str(threshold)
    icas[1].labels_['eyeblink'] = []
    print('Initial Corrmap failed, too many ICA components raised. Initializing new Corrmap; '+thresh_name+'% template match.')
    corrmap(icas=icas, template=(0, 5), threshold = threshold, label = 'eyeblink', plot = True)

In [ ]:
while len(icas[1].labels_['eyeblink']) > 1:
    run_corrmap()

In [ ]:
if len(icas[1].labels_['eyeblink']) == 0:
    raise Exception("No eyeblink components found, raising error.")

#### ICA Report

In [ ]:
ica_copy = ica.copy()

In [ ]:
eog_epochs = mne.preprocessing.create_eog_epochs(raw=raw)
eog_components, eog_scores = ica_copy.find_bads_eog(
    inst=eog_epochs,
    ch_name=['E8','E25'],  # a channel close to the eye
    threshold=1  # lower than the default threshold
)
ica_copy.exclude = eog_components

In [ ]:
ica_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_ica_report.html'
ica_report = mne.Report(title='ICA Filter')
ica_report.add_ica(
    ica=ica_copy,
    title='ICA cleaning',
    picks=icas[1].labels_['eyeblink'],  
    inst=raw,
    eog_evoked=eog_epochs.average(),
    eog_scores=eog_scores,
    n_jobs=1  # could be increased!
)
ica_report_path = os.path.join(preprocessed_report_path, ica_report_title)
ica_report.save(ica_report_path, overwrite=True)

### Create New Raw Data

In [ ]:
icas[1].exclude = icas[1].labels_['eyeblink']

In [ ]:
reconst_raw = raw.copy()
icas[1].apply(reconst_raw)

## Epoch Comparison

### Input Old Epochs

In [ ]:
oldepoch_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_epo.fif'

In [ ]:
oldepoch_file = os.path.join(epoch_report_path, oldepoch_name)
old_epochs = mne.read_epochs(oldepoch_file)
old_epochs

In [ ]:
total_epochs_number = len(old_epochs.events)
total_epochs_number

#### Old Epoch Rejection Filter

In [ ]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

old_epochs.drop_bad(reject=reject_criteria)

In [ ]:
old_epochs_number = len(old_epochs.events)
old_epochs_number

### Run New Epochs

#### Filter Raw Data

In [ ]:
filter_raw = reconst_raw.filter(l_freq=0.1, h_freq=40) #should be updated?
filter_raw

#### Start New Epochs

In [ ]:
tmin = -0.25 #250 ms before the event
tmax = 0.8 #800 ms after the onset

new_epochs = mne.Epochs(reconst_raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    preload=True)
new_epochs

#### New Epoch Rejection Filter

In [ ]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

new_epochs.drop_bad(reject=reject_criteria)

In [ ]:
new_epochs_number = len(new_epochs.events)
new_epochs_number

## Result

## Save and Report New Epochs

In [ ]:
epoch_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_report.html'
epoch_report = mne.Report(title='Preprocessed Epochs')
epoch_report.add_epochs(epochs=new_epochs, title='Preprocessed Epochs')
epoch_report_path = os.path.join(preprocessed_report_path, epoch_report_title)

epoch_report.save(epoch_report_path, overwrite=True)

In [ ]:
preprocessedname = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_epo.fif'

In [ ]:
new_epoch_path = os.path.join(preprocessed_report_path, preprocessedname)
new_epochs.save(new_epoch_path, overwrite=True)